# BFF Experiment: Paper-Accurate Parameters

## Replicating arXiv:2406.19108 Exact Setup

This notebook uses the **exact parameters** from Blaise's published paper:
"Computational Life: How Well-formed, Self-replicating Programs Emerge from Simple Interaction"

### Key Differences from Our Previous Run:

1. **Mutation Rate**: 0.024% (not 0%!)
2. **Soup Size**: 8,192 tapes (not 1,024)
3. **Timing**: Epochs instead of raw interactions
4. **Expected**: 40% transition rate within 16k epochs

### Why Mutations Matter:

Even though Blaise emphasizes zero-mutation evolution via symbiogenesis, the paper actually uses a **tiny mutation rate** (0.024% = 0.00024). This is:
- ~1 byte mutated per 4,167 bytes processed
- Provides "cosmic ray" level perturbation
- Helps break out of local minima
- Still proves symbiogenesis dominates (not mutation-driven)

In [ ]:
# Setup
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from tqdm.notebook import tqdm
import time
from collections import Counter
import json

from core.soup import Soup

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Setup complete!")

## 1. Paper-Accurate Configuration

In [ ]:
# EXACT PARAMETERS FROM PAPER (arXiv:2406.19108)
SOUP_SIZE = 8192          # Paper uses 8,192 tapes
TAPE_LENGTH = 64          # 64 bytes per tape
MUTATION_RATE = 0.00024   # 0.024% mutation rate (KEY DIFFERENCE!)
SEED = 42                 # For reproducibility

# Timing: Paper uses "epochs"
# 1 epoch = all tapes get a chance to interact
# We'll run for 20,000 epochs (paper shows transition by 16k)
TARGET_EPOCHS = 20000
INTERACTIONS_PER_EPOCH = SOUP_SIZE  # Each tape interacts once per epoch
TOTAL_INTERACTIONS = TARGET_EPOCHS * INTERACTIONS_PER_EPOCH

print(f"Configuration (Paper-Accurate):")
print(f"  Soup size: {SOUP_SIZE:,} tapes")
print(f"  Tape length: {TAPE_LENGTH} bytes")
print(f"  Mutation rate: {MUTATION_RATE:.5f} ({MUTATION_RATE*100:.3f}%)")
print(f"  Target epochs: {TARGET_EPOCHS:,}")
print(f"  Total interactions: {TOTAL_INTERACTIONS:,}")
print(f"\n  Paper reports: 40% success rate within 16k epochs")
print(f"  Expected runtime: 15-45 minutes")

## 2. Initialize Soup with Paper Parameters

In [ ]:
print("Initializing primordial soup with paper-accurate parameters...")

soup = Soup(
    size=SOUP_SIZE,
    tape_length=TAPE_LENGTH,
    mutation_rate=MUTATION_RATE,  # Critical: paper uses mutations!
    seed=SEED
)

print(f"\nCreated: {soup}")
print(f"Initial diversity: {soup.get_diversity():.4f}")
print(f"Initial unique tapes: {soup.count_unique_tapes()}/{SOUP_SIZE}")
print(f"\n⚠️  Note: Using {MUTATION_RATE*100:.3f}% mutation rate (not zero!)")

## 3. Run Experiment with Epoch Tracking

Paper methodology:
- Track by **epochs** (not raw interactions)
- 1 epoch = all 8,192 tapes interact once
- Transition expected by epoch 16,000
- We'll run to 20,000 epochs for good measure

In [ ]:
# Run parameters
BATCH_SIZE = 1000
SAMPLE_INTERVAL = SOUP_SIZE  # Sample every epoch

NUM_BATCHES = TOTAL_INTERACTIONS // BATCH_SIZE

# Metrics storage
epoch_numbers = []
sampled_ops_mean = []
sampled_ops_max = []
sampled_diversity = []
sampled_unique_count = []

# Full ops for first 100k for detailed view
full_ops_history = []

# Phase transition detection
transition_detected = False
transition_epoch = None
transition_checkpoint = None

# Performance tracking
start_time = time.time()
last_progress_time = start_time

print(f"Running {TOTAL_INTERACTIONS:,} interactions ({TARGET_EPOCHS:,} epochs)...")
print(f"Sampling every epoch (every {SOUP_SIZE:,} interactions)")
print("\nStarting simulation...\n")

for batch_num in tqdm(range(NUM_BATCHES), desc="Batches"):
    # Run batch
    results = soup.run(num_interactions=BATCH_SIZE, max_ops=10000)
    
    # Store full history early on
    if soup.interaction_count <= 100000:
        full_ops_history.extend([r.operations for r in results])
    
    # Sample metrics every epoch
    if soup.interaction_count % SOUP_SIZE == 0:
        current_epoch = soup.interaction_count // SOUP_SIZE
        batch_ops = [r.operations for r in results]
        
        epoch_numbers.append(current_epoch)
        sampled_ops_mean.append(np.mean(batch_ops))
        sampled_ops_max.append(np.max(batch_ops))
        sampled_diversity.append(soup.get_diversity())
        sampled_unique_count.append(soup.count_unique_tapes())
    
    # Phase transition detection every 100 epochs
    current_epoch = soup.interaction_count // SOUP_SIZE
    if not transition_detected and current_epoch % 100 == 0 and current_epoch > 0:
        if len(sampled_ops_mean) > 10:
            recent_mean = np.mean(sampled_ops_mean[-10:])
            if recent_mean > 500:
                transition_detected = True
                transition_epoch = current_epoch
                transition_checkpoint = soup.get_state()
                print(f"\n🎉 PHASE TRANSITION at epoch {transition_epoch:,}!")
                print(f"   Operations: {recent_mean:.1f} ops/interaction")
                print(f"   Diversity: {soup.get_diversity():.4f}")
    
    # Progress every 1000 epochs
    if current_epoch % 1000 == 0 and current_epoch > 0:
        elapsed = time.time() - last_progress_time
        interactions_per_sec = (1000 * SOUP_SIZE) / elapsed
        last_progress_time = time.time()
        
        print(f"\n[Epoch {current_epoch:,}]")
        print(f"  Speed: {interactions_per_sec:.1f} int/sec")
        if len(sampled_ops_mean) > 0:
            print(f"  Ops: {sampled_ops_mean[-1]:.1f} mean, {sampled_ops_max[-1]:.0f} max")
            print(f"  Diversity: {soup.get_diversity():.4f}")

total_time = time.time() - start_time
final_epoch = soup.interaction_count // SOUP_SIZE

print(f"\n{'='*70}")
print(f"SIMULATION COMPLETE")
print(f"{'='*70}")
print(f"Total epochs: {final_epoch:,}")
print(f"Total interactions: {soup.interaction_count:,}")
print(f"Runtime: {total_time/60:.1f} minutes")
print(f"Speed: {soup.interaction_count/total_time:.1f} int/sec")
print(f"\nFinal state: {soup}")
print(f"Final diversity: {soup.get_diversity():.4f}")

if transition_detected:
    print(f"\n✅ Phase transition at epoch {transition_epoch:,}")
    print(f"   Within paper's 16k epoch window: {transition_epoch <= 16000}")
else:
    print(f"\n⚠️  No transition (max ops: {max(sampled_ops_mean) if sampled_ops_mean else 0:.1f})")
    print(f"   Paper reports 40% success rate - try different seed!")

## 4. Visualize Results (Epoch-Based)

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# Plot 1: Operations vs Epoch
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(epoch_numbers, sampled_ops_mean, 'b-', linewidth=1.5, label='Mean ops', alpha=0.7)
ax1.plot(epoch_numbers, sampled_ops_max, 'r-', linewidth=1, label='Max ops', alpha=0.5)
ax1.set_xlabel('Epoch Number', fontsize=12)
ax1.set_ylabel('Operations per Interaction', fontsize=12)
ax1.set_title('BFF Abiogenesis: Paper-Accurate Parameters (arXiv:2406.19108)', 
              fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper left')

# Mark 16k epoch threshold from paper
ax1.axvline(x=16000, color='orange', linestyle=':', linewidth=2, 
            label='Paper threshold (16k epochs)', alpha=0.5)

if transition_detected:
    ax1.axvline(x=transition_epoch, color='green', linestyle='--', linewidth=2, 
                label=f'Transition (epoch {transition_epoch:,})')
ax1.legend(loc='upper left')

# Plot 2: Diversity
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(epoch_numbers, sampled_diversity, 'g-', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=10)
ax2.set_ylabel('Diversity', fontsize=10)
ax2.set_title('Population Diversity', fontsize=12)
ax2.set_ylim([0, 1.05])
ax2.grid(True, alpha=0.3)
if transition_detected:
    ax2.axvline(x=transition_epoch, color='red', linestyle='--', alpha=0.5)

# Plot 3: Unique count
ax3 = fig.add_subplot(gs[1, 1])
ax3.plot(epoch_numbers, sampled_unique_count, 'purple', linewidth=2)
ax3.axhline(y=SOUP_SIZE, color='gray', linestyle=':', label=f'Total ({SOUP_SIZE})')
ax3.set_xlabel('Epoch', fontsize=10)
ax3.set_ylabel('Unique Tapes', fontsize=10)
ax3.set_title('Replicator Takeover', fontsize=12)
ax3.grid(True, alpha=0.3)
ax3.legend()
if transition_detected:
    ax3.axvline(x=transition_epoch, color='red', linestyle='--', alpha=0.5)

plt.savefig('../experiments/run_paper_params.png', dpi=150, bbox_inches='tight')
print("Figure saved to: experiments/run_paper_params.png")
plt.show()

## 5. Analysis: Compare to Paper Results

In [ ]:
print("="*70)
print("COMPARISON TO PAPER (arXiv:2406.19108)")
print("="*70)

print("\nPaper Metrics:")
print("  Success rate: 40% within 16k epochs")
print("  Configuration: 8,192 tapes, 64 bytes, 0.024% mutation")

print("\nOur Results:")
print(f"  Soup size: {SOUP_SIZE:,} tapes ✓")
print(f"  Tape length: {TAPE_LENGTH} bytes ✓")
print(f"  Mutation rate: {MUTATION_RATE*100:.3f}% ✓")
print(f"  Epochs run: {final_epoch:,}")

if transition_detected:
    print(f"\n  ✅ TRANSITION DETECTED at epoch {transition_epoch:,}")
    within_window = "✅ YES" if transition_epoch <= 16000 else "❌ NO"
    print(f"  Within 16k epoch window: {within_window}")
    print(f"  Final mean ops: {sampled_ops_mean[-1]:.1f}")
else:
    print(f"\n  ❌ No transition detected")
    print(f"  Maximum mean ops: {max(sampled_ops_mean) if sampled_ops_mean else 0:.1f}")
    print(f"\n  This is within expected variance (60% no-transition rate)")
    print(f"  Recommendations:")
    print(f"    - Try different seed")
    print(f"    - Run longer (30k epochs)")
    print(f"    - Check for weak signals (ops > 100)")

# Population analysis
tape_hashes = soup.get_tape_hashes()
hash_counts = Counter(tape_hashes)

print(f"\nPopulation State:")
print(f"  Unique tapes: {len(hash_counts)}/{SOUP_SIZE}")
print(f"  Diversity: {soup.get_diversity():.4f}")

if len(hash_counts) > 0:
    top1 = hash_counts.most_common(1)[0][1]
    print(f"  Dominant replicator: {top1} copies ({100*top1/SOUP_SIZE:.1f}%)")
    
    if top1 > SOUP_SIZE * 0.05:  # >5% means replication happening
        print(f"\n  ⚠️  Replication detected (even without full transition)!")

## 6. Save Results

In [ ]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Save metrics
metrics_path = Path(f'../experiments/paper_params_{timestamp}.json')
metrics = {
    'config': {
        'soup_size': SOUP_SIZE,
        'tape_length': TAPE_LENGTH,
        'mutation_rate': MUTATION_RATE,
        'seed': SEED,
        'target_epochs': TARGET_EPOCHS,
        'source': 'arXiv:2406.19108 parameters'
    },
    'results': {
        'runtime_seconds': total_time,
        'final_epoch': final_epoch,
        'transition_detected': transition_detected,
        'transition_epoch': transition_epoch,
        'final_diversity': soup.get_diversity(),
        'final_unique_count': soup.count_unique_tapes(),
    },
    'time_series': {
        'epoch_numbers': epoch_numbers,
        'sampled_ops_mean': sampled_ops_mean,
        'sampled_ops_max': sampled_ops_max,
        'sampled_diversity': sampled_diversity,
        'sampled_unique_count': sampled_unique_count,
    }
}

with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"Metrics saved: {metrics_path}")

# Save final state
state_path = Path(f'../experiments/checkpoints/paper_params_{timestamp}.json')
with open(state_path, 'w') as f:
    json.dump(soup.get_state(), f)
print(f"State saved: {state_path}")

if transition_detected and transition_checkpoint:
    trans_path = Path(f'../experiments/checkpoints/paper_transition_{timestamp}.json')
    with open(trans_path, 'w') as f:
        json.dump(transition_checkpoint, f)
    print(f"Transition checkpoint saved: {trans_path}")

print("\n✅ All results saved!")

## Summary

This notebook uses the **exact parameters from the published paper**:

### Key Corrections:
1. **Mutation rate**: 0.024% (not 0%)
2. **Soup size**: 8,192 tapes (larger population)
3. **Epoch tracking**: Proper alignment with paper methodology

### Why Previous Run Didn't Transition:
- **Zero mutations**: Made it too stable (harder to escape)
- **Smaller soup**: 1,024 vs 8,192 reduces probability
- **Not enough time**: Need more epochs

### Expected Behavior:
- **40% chance** of transition by epoch 16,000
- **60% chance** of no transition (need different seed)
- Mutations provide "kicks" to explore state space
- Still symbiogenesis-dominated (not mutation-driven evolution)

The tiny mutation rate is **not** driving evolution - it's providing occasional perturbations that help the system find replicating configurations. Once found, symbiogenesis takes over and dominates the evolutionary dynamics!